# Amazon Braket Spending Limits: An Introduction

## Introduction

Amazon Braket spending limits provide essential budget control for quantum computing experiments. This feature helps prevent unexpected charges by allowing you to set maximum spending thresholds for specific quantum devices. This notebook provides a hands-on guide to help you get started with spending limits and experiment with their configuration and functionality.

### Key Characteristics
- **Device-specific control**: Each quantum device either has a spending limit or operates without restrictions
- **Cumulative tracking**: Total spending is monitored across all tasks on a device over time
- **Dual validation**: Tasks are checked against both budget and time constraints before execution
- **Automatic enforcement**: Non-compliant tasks are rejected immediately, preventing unexpected costs
- **Flexible management**: You can update, delete, or create new spending limits at any time

📖 For more information about spending limits and task validation, see the [Amazon Braket Developer Guide - Spending Limits](https://docs.aws.amazon.com/braket/latest/developerguide/braket-pricing.html).

### Why Use Spending Limits
- **Budget Management**: Control costs during experimentation and development
- **Educational Use**: Prevent accidental overspending in learning environments
- **Production Safety**: Add guardrails for automated quantum workflows
- **Team Collaboration**: Ensure shared resources stay within allocated budgets

### Configuration Options
Currently, the Amazon Braket SDK does not provide wrappers for interacting with the spending limits. As such, there are three ways to access Braket to configure spending limits:
1. **AWS Console** - Web-based graphical interface
2. **AWS CLI** - Command-line interface for configuration
3. **AWS SDK (boto3)** - Programmatic access from Python applications

This notebook uses `boto3` for programmatic API access. Equivalent AWS CLI commands are provided for reference throughout the demonstration.

In [1]:
# Import boto3 for programmatic API access
import boto3

### ⚠️ Important Notice
This notebook will create or temporarily modify spending limits for demonstration purposes. This may interfere with other quantum computing tasks running from the same AWS account. Please coordinate with your team if you share an AWS account for quantum computing workloads.

In [2]:
# Import required libraries
import json
import pprint

from braket.aws import AwsDevice
from braket.aws.queue_information import QueueType
from braket.circuits import Circuit
from braket.devices import Devices

# Configuration constants
DEMO_SPENDING_LIMIT = 3.0  # Demo budget in USD
HIGH_COST_SHOTS = 10000    # Shot count that exceeds our budget
LOW_COST_SHOTS = 500       # Shot count within our budget
MAX_QUEUE_THRESHOLD = 10   # Maximum acceptable queue depth

## Choosing your QPU Device
Amazon Braket allows users to set different spending limits for different quantum devices supported by the service. 

We'll use Rigetti's Ankaa-3 device for this demonstration. To select a different device - change the ARN in the cell below.

In [3]:
device_arn = Devices.Rigetti.Ankaa3

In [4]:
# Select and initialize the quantum device
region = AwsDevice.get_device_region(device_arn)
device = AwsDevice(device_arn)

print(f"🔧 Selected Device: {device_arn}")
print(f"🌍 Region: {region}")
print(f"📊 Device Status: {'Available' if device.is_available else 'Unavailable'}")

# Display current queue information
queue_depth = device.queue_depth()
print(f"📋 Queue Status: {queue_depth.jobs} total jobs")
print(f"   - Normal priority: {queue_depth.quantum_tasks[QueueType.NORMAL]}")
print(f"   - High priority: {queue_depth.quantum_tasks[QueueType.PRIORITY]}")

🔧 Selected Device: arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3
🌍 Region: us-west-1
📊 Device Status: Unavailable
📋 Queue Status: 0 total jobs
   - Normal priority: 2
   - High priority: 0


## Managing spending limits

### Searching for existing spending limits

First let us see if there is already a spending limit set for the device.

In [5]:
# Configure search filters for the specific device
search_filters = [
    {
        'name': 'deviceArn',
        'values': [device_arn],
        'operator': 'EQUAL'
    }
]

# Initialize Braket client
braket_client = boto3.client("braket", region_name=region)

print("🔍 Searching for existing spending limits...\n")
print("📋 Equivalent AWS CLI command:")
print(f"aws --region {region} braket search-spending-limits --filters='{json.dumps(search_filters)}'\n")

# Execute the search
response = braket_client.search_spending_limits(filters=search_filters)
spending_limits = response['spendingLimits']

🔍 Searching for existing spending limits...

📋 Equivalent AWS CLI command:
aws --region us-west-1 braket search-spending-limits --filters='[{"name": "deviceArn", "values": ["arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3"], "operator": "EQUAL"}]'



In [6]:
# Process search results
existing_limit_amount = None
existing_spending_limit = None

if len(spending_limits) > 0:
    # Should only be one limit per device
    assert len(spending_limits) == 1, f"Expected 1 limit, found {len(spending_limits)}"
    
    existing_spending_limit = spending_limits[0]
    existing_limit_amount = existing_spending_limit['spendingLimit']
    
    print(f"💰 Existing spending limit: ${float(existing_limit_amount):.2f}")
    print("📊 Current usage:")
    print(f"   - Total spent: ${float(existing_spending_limit.get('totalSpend', '0')):.2f}")
    print(f"   - Queued spend: ${float(existing_spending_limit.get('queuedSpend', '0')):.2f}")
    print("\n📋 Full details:")
    pprint.pprint(existing_spending_limit)
else:
    print("🆕 No existing spending limit found for this device")
    print("   We'll create a new one in the next step")

🆕 No existing spending limit found for this device
   We'll create a new one in the next step


## Creating / Updating Spending Limits

Now we will allocate a \$3 budget for ourselves. We do that by either updating the existing spending limit or by creating a new one if no existing spending limit was found.

In [7]:
def calculate_safe_limit(existing_limit):
    """
    Calculate a safe spending limit that accounts for existing usage.
    
    Args:
        existing_limit: Dictionary containing existing limit information
        
    Returns:
        float: Safe limit amount that won't immediately conflict
    """
    if not existing_limit:
        return DEMO_SPENDING_LIMIT
    
    total_spent = float(existing_limit.get('totalSpend', 0))
    queued_spend = float(existing_limit.get('queuedSpend', 0))
    current_usage = total_spent + queued_spend
    
    return current_usage + DEMO_SPENDING_LIMIT

# Determine the appropriate action and limit amount
new_limit_amount = calculate_safe_limit(existing_spending_limit)

if not existing_limit_amount:
    # Create new spending limit
    print(f"🆕 Creating new spending limit: ${new_limit_amount:.2f}\n")
    print("📋 Equivalent AWS CLI command:")
    print(f"aws --region {region} braket create-spending-limit "
          f"--device-arn {device_arn} "
          f"--spending-limit {new_limit_amount:.2f}\n")
    
    response = braket_client.create_spending_limit(
        deviceArn=device_arn, 
        spendingLimit=f"{new_limit_amount:.2f}"
    )
    spending_limit_arn = response['spendingLimitArn']        
else:
    # Update existing spending limit
    spending_limit_arn = existing_spending_limit['spendingLimitArn']
    
    print(f"🔄 Updating existing spending limit from ${float(existing_limit_amount):.2f} "
           f"to ${new_limit_amount:.2f}\n")
    print("📋 Equivalent AWS CLI command:")
    print(f"aws --region {region} braket update-spending-limit "
          f"--spending-limit-arn {spending_limit_arn} "
          f"--spending-limit {new_limit_amount:.2f}\n")
    
    response = braket_client.update_spending_limit(
        spendingLimitArn=spending_limit_arn, 
        spendingLimit=f"{new_limit_amount:.2f}"
    )

🆕 Creating new spending limit: $3.00

📋 Equivalent AWS CLI command:
aws --region us-west-1 braket create-spending-limit --device-arn arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3 --spending-limit 3.00



## Testing Cost Controls

We will now attempt to submit quantum tasks, using a simple circuit (state-preparation for a uniform superposition of two qubits). We will control the cost of the tasks by changing the number of shots we wish to perform.

**NOTE**: We chose to modify the cost of task execution for the purpose of demonstrating the real-time spending limit enforcement. One could also experiment with modifying the time period of the spending limit (for example, we can set it to start one day in the future by updating the `["timePeriod"]["startAt"]` entry of the spending limit and setting it to `(datetime.datetime.now() + datetime.timedelta(days=1)).isoformat()`).

In [8]:
# Create a simple test circuit
test_circuit = Circuit().h(0).h(1)

print("Test Circuit:")
print(test_circuit)

Test Circuit:
T  : │  0  │
      ┌───┐ 
q0 : ─┤ H ├─
      └───┘ 
      ┌───┐ 
q1 : ─┤ H ├─
      └───┘ 
T  : │  0  │


### Task outside of our budget

In [9]:
print("\n💰 Cost Analysis:")
print("   - Per-shot cost: $0.0009")
print("   - Per-task cost: $0.30")
print(f"   - Requested shots: {HIGH_COST_SHOTS:,}")
print(f"   - Estimated cost: ~${0.30 + HIGH_COST_SHOTS * 0.0009:.2f}")
print(f"   - Available budget: ${DEMO_SPENDING_LIMIT:.2f}")
print("   - Expected result: ❌ Task should be rejected")


💰 Cost Analysis:
   - Per-shot cost: $0.0009
   - Per-task cost: $0.30
   - Requested shots: 10,000
   - Estimated cost: ~$9.30
   - Available budget: $3.00
   - Expected result: ❌ Task should be rejected


In [10]:
# Attempt to submit the high-cost task
print(f"🚀 Attempting to submit task with {HIGH_COST_SHOTS:,} shots...\n")

try:
    job = device.run(test_circuit, shots=HIGH_COST_SHOTS)
    print(f"⚠️  Unexpected: Task was accepted! Job ID: {job.id}")
    print("   This might indicate the spending limit isn't active yet")
    
except Exception as e:
    print("✅ Expected behavior: Task was rejected.")
    print("\n📋 Exception details:")
    print(f"   Type: {type(e).__name__}")
    print(f"   Message: {str(e)}")
    
    # Check if it's specifically a spending limit error
    if "spending limit" in str(e).lower():
        print("\n🎯 Confirmed: This is a spending limit enforcement")
    else:
        print("\n❓ Note: This might be a different type of error")

🚀 Attempting to submit task with 10,000 shots...

✅ Expected behavior: Task was rejected.

📋 Exception details:
   Type: ValidationException
   Message: An error occurred (ValidationException) when calling the CreateQuantumTask operation: Attempted to spend over the spending limit or outside the spending limit time window. Spending limit: 3, current spend: 0, estimated cost: 9.3000000000, start time: 2025-11-17T20:42:42.985Z, end time: 2125-12-30T00:00:00.000Z

🎯 Confirmed: This is a spending limit enforcement


### Task within our budget

In [11]:
# Test with a lower shot count that should be within budget
low_cost_estimate = 0.30 + LOW_COST_SHOTS * 0.0009

print("🧪 Testing with acceptable cost:")
print(f"   - Shots: {LOW_COST_SHOTS:,}")
print(f"   - Estimated cost: ~${low_cost_estimate:.2f}")
print("   - Expected result: ✅ Should be accepted\n")

🧪 Testing with acceptable cost:
   - Shots: 500
   - Estimated cost: ~$0.75
   - Expected result: ✅ Should be accepted



⚠️ *Note*: Executing the following code will submit a task incurring a cost. To make sure this is not done unintentionally the cell below is in markdown by default.

```python
# Check device availability first
if device.is_available:
    queue_depth = device.queue_depth()
    total_queue = (int(queue_depth.quantum_tasks[QueueType.NORMAL]) + 
                   int(queue_depth.quantum_tasks[QueueType.PRIORITY]))
    
    if total_queue < MAX_QUEUE_THRESHOLD:
        print(f"📊 Device status: Available (queue: {total_queue} tasks)")
        print("🚀 Submitting low-cost task...\n")
        
        try:
            job = device.run(test_circuit, shots=LOW_COST_SHOTS)
            print(f"✅ Success! Task accepted with Job ID: {job.id}")
            print(f"📊 Task status: {job.state()}")
            print("\n💡 This confirms spending limits allow reasonable usage")
            
        except Exception as e:
            print(f"❌ Unexpected: Low-cost task was also rejected")
            print(f"   Error: {e}")
            print("   This might indicate a configuration issue")
    else:
        print(f"⏳ Device queue is busy ({total_queue} tasks) - skipping submission")
        print("   You can try again when the queue is less busy")
else:
    print("🔒 Device is currently unavailable - cannot test task submission")
    print("   This is normal and doesn't affect spending limit functionality")
```

## Cleanup

It's important to clean up the changes made during this demonstration. We'll either:
- **Delete** the spending limit if we created a new one
- **Restore** the original limit if we updated an existing one

In [12]:
# Cleanup: Restore original state
if not existing_limit_amount:
    # Delete the spending limit we created
    print("🗑️  Deleting the spending limit we created\n")
    print("📋 Equivalent AWS CLI command:")
    print(f"aws --region {region} braket delete-spending-limit "
          f"--spending-limit-arn {spending_limit_arn}\n")
    
    response = braket_client.delete_spending_limit(
        spendingLimitArn=spending_limit_arn
    )
    print("🔄 Device spending is now unrestricted")
        
else:
    # Restore the original spending limit
    print(f"🔄 Restoring original spending limit: ${float(existing_limit_amount):.2f}\n")
    print("📋 Equivalent AWS CLI command:")
    print(f"aws --region {region} braket update-spending-limit "
          f"--spending-limit-arn {spending_limit_arn} "
          f"--spending-limit {existing_limit_amount}\n")
    
    response = braket_client.update_spending_limit(
        spendingLimitArn=spending_limit_arn, 
        spendingLimit=existing_limit_amount
    )

🗑️  Deleting the spending limit we created

📋 Equivalent AWS CLI command:
aws --region us-west-1 braket delete-spending-limit --spending-limit-arn arn:aws:braket:us-west-1:012345678910:spending-limit/57bd9d3c-c6b3-4e4a-b6a0-ad9380f9f097

🔄 Device spending is now unrestricted


## Next Steps

Now that you understand the basics of spending limits, consider exploring:

- [Amazon Braket Pricing](https://aws.amazon.com/braket/pricing/)
- [Amazon Braket Developer Guide](https://docs.aws.amazon.com/braket/)
- [Amazon Braket Examples](https://github.com/aws/amazon-braket-examples)
- [AWS Cost Management](https://aws.amazon.com/aws-cost-management)

---

**📝 Note**: This notebook demonstrated spending limit management in a safe, controlled manner. Always review costs and limits before running production quantum workloads.